## Extracting Transforming Load (ETL)

En este notebook se planea procesar los datos encontrados en los archivos .csv. Con este fin vamos a usar distintas librerias que nos permitan obtener una base de datos limpia. Es de tener en cuenta que se usaran la menor cantidad de celdas de accion con el fin de hacer nuestro codigo lo mas corto y simple posible. De ser necesario, las interacciones tendran su respectivo comentario que nos ayudara a entender el codigo.

Nuestro codigo se repartira en distintas secciones teniendo una similitud con el patron Modelo - Vista - Controlador (MVC):

Celda de librerias.
Celda de acciones.
Celda de vistas.


## 1. Librerias
Esta Celda nos ayudara a traer todas las librerias necesarias con el fin de completar el ejercicio

In [9]:
#Importamos las librerias necesarias para el ejercicio

import pandas as pd
import numpy as np

#Ignoramos los warnings
import warnings
warnings.filterwarnings("ignore")


## 2. Acciones
Esta celda nos ayudara a crear todas las funciones necesarias

In [10]:
#Funcion que crea diccionarios para poder contar los tipos de datos que se encuentran en cada dataframe
def diccionario_tipos_datos(dataframe_file):

    dicc_col = {}
    
    for columna in dataframe_file:
        conteo_tipos = dataframe_file[columna].apply(type).value_counts().to_dict()
        dicc_col[columna] = {str(tipo): conteo for tipo, conteo in conteo_tipos.items()}

    return dicc_col

#Funcion que convierte los datos a flotantes si son numeros o ceros si son de otro tipo

def cambio_a_float(dato):

    if pd.isna(dato):
        return 0.0
    
    try:
        return float(dato)
    except(ValueError, TypeError):
        return 0.0

#Funcion que convierte los datos a Datetime y devuelve None en caso de ser otro tipo

def cambio_a_datetime(dato):

    if pd.isna(dato):
        return None

    try:
        return pd.to_datetime(dato)
    except ValueError:
        return None

## 3. Vistas
## 3.1. Obtencion de datos

En esta seccion nos enfocamos en transformar los datos del formato original (.csv) a un formato Dataframe de pandas con el fin de poder observar, eliminar, transformar y demas procesos que nos permitan una mejor visualizacion y optimizacion de dichos datos.

Los procesos a aplicar son:

* Transformacion de .csv a Dataframe

* Eliminacion y agregacion de columnas

* Eliminacion de valores Null/Nan en los distintos Datasets

* Reinicio de indices

* Casting de datos a formatos correspondientes



## 3.1.1 ETL para el Dataset: "Internet_Penetracion.csv"

In [11]:
#Carga del dataset
inter_hogar = pd.read_csv("C:\\Users\\andre\\Documents\\Py Codes\\data\\Internet_Penetracion.csv", sep=",")
print("Carga completa del dataset: Internet_Penetracion.csv")

#Remplazo de "," a "." en columnas: Accesos por cada 100 hogares; Accesos por cada 100 hab
col_trabajar = ["Accesos por cada 100 hogares","Accesos por cada 100 hab"]
inter_hogar["Accesos por cada 100 hogares"] = inter_hogar["Accesos por cada 100 hogares"].str.replace(",", ".")
inter_hogar["Accesos por cada 100 hab"] = inter_hogar["Accesos por cada 100 hab"].str.replace(",", ".")
print(f"Caracteres cambiados en columnas: {col_trabajar}")

#Casting str a float
inter_hogar["Accesos por cada 100 hogares"] = inter_hogar["Accesos por cada 100 hogares"].apply(cambio_a_float)
inter_hogar["Accesos por cada 100 hab"] = inter_hogar["Accesos por cada 100 hab"].apply(cambio_a_float)
print(f"Cambio format a float en columnas: {col_trabajar}")

#Borrado de datos null
inter_hogar.dropna(inplace=True)
print("Datos vacios eliminados")

Carga completa del dataset: Internet_Penetracion.csv
Caracteres cambiados en columnas: ['Accesos por cada 100 hogares', 'Accesos por cada 100 hab']
Cambio format a float en columnas: ['Accesos por cada 100 hogares', 'Accesos por cada 100 hab']
Datos vacios eliminados


## 3.1.2 ETL para el Dataset: "Internet_BAF.csv"

In [13]:
#Carga del dataset
inter_tech = pd.read_csv("C:\\Users\\andre\\Documents\\Py Codes\\data\\Internet_BAF.csv",sep=",")

#Modificacion de la col "total"
inter_tech.drop(columns="Total", inplace=True)
inter_tech["total"] = inter_tech["Banda ancha fija"] + inter_tech["Dial up"]
print("Columna: Total modificada")

#Borrado de datos null
inter_tech.dropna(inplace=True)
print("Datos vacios eliminados")

Columna: Total modificada
Datos vacios eliminados


## 3.1.3 ETL para el Dataset: "Internet_Accesos-por-conectividad.csv"

In [14]:
#Carga del dataset
inter_acess = pd.read_csv("C:\\Users\\andre\\Documents\\Py Codes\\data\\Internet_Accesos-por-conectividad.csv",sep=",")

#Casting a Int a diferentes columnas

col_trabajar = ["Año", "Trimestre"]
inter_acess["Año"] = inter_acess["Año"].apply(cambio_a_float).astype(int)
inter_acess["Trimestre"] = inter_acess["Trimestre"].apply(cambio_a_float).astype(int)
print(f"Cambio a Format int en columnas: {col_trabajar}")

#Modificacion columna "Total"
inter_acess.drop(columns="Total",inplace=True)
inter_acess["Total"] = inter_acess["ADSL"] + inter_acess["Cablemodem"] + inter_acess["Fibra óptica"] + inter_acess["Wireless"]+ inter_acess["Otros"]
print(f"Columna: Total Modificada")

#Borrado de datos null
inter_acess.dropna(inplace=True)
print("Datos vacios eliminados")

Cambio a Format int en columnas: ['Año', 'Trimestre']
Columna: Total Modificada
Datos vacios eliminados


## 4. Revision Tipo de Datos

Con la funcion definida anterior como "diccionario_tipos_datos(dataframe_file)" revisamos los tipos de datos que tienen cada una de las columnas dentro de nuestras bases de datos.


In [15]:
diccionario_tipos_datos(inter_hogar)

{'Año': {"<class 'int'>": 36},
 'Trimestre': {"<class 'int'>": 36},
 'Accesos por cada 100 hogares': {"<class 'float'>": 36},
 'Accesos por cada 100 hab': {"<class 'float'>": 36},
 'Periodo': {"<class 'str'>": 36}}

In [16]:
diccionario_tipos_datos(inter_tech)

{'Año': {"<class 'int'>": 862},
 'Trimestre': {"<class 'int'>": 862},
 'Provincia': {"<class 'str'>": 862},
 'Banda ancha fija': {"<class 'int'>": 862},
 'Dial up': {"<class 'float'>": 862},
 'total': {"<class 'float'>": 862}}

In [17]:
diccionario_tipos_datos(inter_acess)

{'Año': {"<class 'int'>": 864},
 'Trimestre': {"<class 'int'>": 864},
 'Provincia': {"<class 'str'>": 864},
 'ADSL': {"<class 'float'>": 864},
 'Cablemodem': {"<class 'float'>": 864},
 'Fibra óptica': {"<class 'float'>": 864},
 'Wireless': {"<class 'float'>": 864},
 'Otros': {"<class 'float'>": 864},
 'Total': {"<class 'float'>": 864}}

## 5. Actualizacion de Datasets

En esta seccion actualizaremos los datasets trabajados. Esto nos permitira tener los datos en los formatos adecuados y limpios.

In [18]:
inter_hogar.to_csv("C:\\Users\\andre\\Documents\\Py Codes\\data\\Internet_Penetracion.csv", index=False)
print("Dataset Actualizado: inter_hogar")

inter_tech.to_csv("C:\\Users\\andre\\Documents\\Py Codes\\data\\Internet_BAF.csv",index=False)
print("Dataset Actualizado: inter_tech")

inter_acess.to_csv("C:\\Users\\andre\\Documents\\Py Codes\\data\\Internet_Accesos-por-conectividad.csv",index=False)
print("Dataset Actualizado: inter_acess")


Dataset Actualizado: inter_hogar
Dataset Actualizado: inter_tech
Dataset Actualizado: inter_acess
